In [17]:
import os
os.chdir('../scripts')

from scrape_domain import *
from constants import *
from utils import *

In [18]:
#find pattern for week/annual rent
year_pattern = r"p.a.|annually|annual"
week_pattern = r"pw|week"

regex_week_pattern = re.compile(week_pattern)
regex_year_pattern = re.compile(year_pattern)


def remove_chars(s):
    return re.sub('[^0-9.]+', '', s) 

#flag type of rent with 1(weekly), 0(annul), None(unknown)
def type_flag(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return 1
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return 0
    # not speficied
    else:
        return None

#convert rent to weekly rent
#if not sure rent type, it will be considered as weekly rent
def convert_rent(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return remove_chars(s)
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return float(remove_chars(s))//12
    # not speficied
    else:
        return remove_chars(s)

In [19]:

#input url return dict
def url_to_df(url_list):
    
    recent_rented_number_list = []
    street_renter_owner_ratio_list = []
    neighb_family_single_ratio_list = []
    long_term_resi_percentage_list =[]
    price_list = []
    
    
    for url in url_list  :
        
        link = url
        
        # opening urls
        bs_object = BeautifulSoup(requests.get(link, headers=headers).text, "html.parser")

        #recent rented
        recent_rented_number = bs_object.find("div", {"class": "css-12uo2x5"})

        #neighbourhood family vs single ratio
        neighb_family = bs_object.find("span", {"class": "css-1ruej2h"})
        neighb_single = bs_object.find("span", {"class": "css-1ymya4c"})


        #street owner vs renter
        street_renter = bs_object.find("span", {"class": "css-1ejmyle"})
        street_owner = bs_object.find("span", {"class": "css-1jhlvvv"})


        #long term resident percentage
        long_term_resi= bs_object.find("div", {"class": "css-ibsnk8"})

        # extracting property price
        property_price = bs_object.find("div", {"data-testid": "listing-details__summary-title"})



        # dictionary to store temporary data
        pattern1 = re.compile(r'>(.+)<.')

        # putting 'none' if recent_rented number is missing   
        if recent_rented_number is None:
            recent_rented_number_list.append(None)

        else:
            recent_rented_number_list.append(pattern1.findall(str(recent_rented_number))[0])



        # putting 'none' if longterm resident number is missing   
        if long_term_resi is None:
            long_term_resi_percentage_list.append(None)

        else:
            long_term_resi_percentage_list.append(float(str(long_term_resi.text[:-1]))/100)



        # putting 'none' if neighbourhood family/single number is missing   
        if (neighb_family is None) |(neighb_single is None) :
            neighb_family_single_ratio_list.append(None)

        else:
            #to avoid zero division
            add_on = 0.000001
            neighb_family_single_ratio_list.append(float(str(neighb_family.text[:-1]))/(float(str(neighb_single.text[:-1]))+add_on))




        # putting 'none' if street owener/renter number is missing   
        if (street_owner is None) | (street_renter is None):
            street_renter_owner_ratio_list.append(None)

        else:
            #to avoid zero division
            add_on = 0.000001
            street_renter_owner_ratio_list.append(float(str(street_renter.text[:-1]))/(float(str(street_owner.text[:-1]))+add_on))



        # putting 'none' if price is missing    
        if property_price is None:
            price_list.append(None)

        else:
            price_list.append(pattern1.findall(str(property_price))[0])
            
         # appending all the data into a list
        
    house_dict = {}    
    house_dict['Rent'] = price_list
    house_dict['street_renter_owner_ratio'] = street_renter_owner_ratio_list 
    house_dict['neighb_family_single_ratio'] = neighb_family_single_ratio_list
    house_dict['long_term_resi_percentage'] = long_term_resi_percentage_list
    house_df = pd.DataFrame(house_dict)
    
    house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
    house_df['Type'] = [type_flag(i) for i in price_list]
    house_df['url'] = [i for i in url_list]
    
    return house_df    
        
        

In [4]:
'''def list_to_df(basic_feature_list):
    
    recent_rented_number_list = []
    street_renter_owner_ratio_list = []
    neighb_family_single_ratio_list = []
    long_term_resi_percentage_list =[]
    price_list = []

    for row in basic_feature_list:
        # checking if the row cointains 'price'         
        if 'price' in row:
            price_list.append(row['price'])
        else:
            price_list.append(None)        

        # checking if the row cointains 'recent_rented_number'            
        if 'recent_rented_number' in row:
            recent_rented_number_list.append(row['recent_rented_number'])
        else:
            recent_rented_number_list.append(None)


        # checking if the row cointains 'long_term_resi_percentage'            
        if 'long_term_resi_percentage' in row:
            long_term_resi_percentage_list.append(row['long_term_resi_percentage'])
        else:
            long_term_resi_percentage_list.append(None)

        # checking if the row cointains 'street_renter_owner_ratio'            
        if 'street_renter_owner_ratio' in row:
            street_renter_owner_ratio_list.append(row['street_renter_owner_ratio'])
        else:
            street_renter_owner_ratio_list.append(None)

        # checking if the row cointains 'neighb_family_single_ratio'            
        if 'neighb_family_single_ratio' in row:
            neighb_family_single_ratio_list.append(row['neighb_family_single_ratio'])
        else:
            neighb_family_single_ratio.append(None)
        
        
    house_dict = {}    
    house_dict['Rent'] = price_list
    house_dict['street_renter_owner_ratio'] = street_renter_owner_ratio_list 
    house_dict['neighb_family_single_ratio'] = neighb_family_single_ratio_list
    house_dict['long_term_resi_percentage'] = long_term_resi_percentage_list
    house_df = pd.DataFrame(house_dict)
    
    house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
    house_df['Type'] = [type_flag(i) for i in price_list]
    return house_df'''



In [20]:
vic_postcodes = constants.postcodes['VIC_sample']

domain_links = set()

for i in tqdm(range(len(vic_postcodes))):
    current_links = domain_property_links(vic_postcodes[i])
    domain_links = domain_links | current_links
    
    print(len(current_links))

  0%|                                                   | 0/100 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [6]:
with open('2022-09-05.VIC.domain.pickle', 'wb') as file:
    pickle.dump(domain_links, file , protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('2022-09-04.VIC.domain.pickle', 'rb') as file:
    links = pickle.load(file)

In [22]:
test_links = list(links)
len(test_links)

15353

In [ ]:
df= url_to_df(test_links)

In [ ]:
df